In [1]:
import pandas as pd
import numpy as np
import torch
import cv2
import mediapipe as mp
import requests

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [2]:
words_corpus_df = pd.read_csv("augmented_df.csv")

In [3]:
words_corpus_df = words_corpus_df[["Word", "Frames path"]]

In [4]:
words_corpus_df.head()

,Word,Frames path
0,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
1,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
2,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
4,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...


In [5]:
words_corpus_df.tail()

,Word,Frames path
3187,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3188,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3189,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3190,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3191,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...


In [6]:
words_corpus_df.isna().sum()

Word           0
Frames path    0
dtype: int64

In [7]:
words_corpus_df.shape

(3192, 2)

In [8]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

def extract_features(hand_landmarks):
    # Extracting the x, y, z coordinates of the landmarks
    features = []
    for landmark in hand_landmarks.landmark:
        features.append(landmark.x)
        features.append(landmark.y)
        features.append(landmark.z)

    # Convert list to a numpy array (flatten the landmarks)
    features = np.array(features)
    
    # Optionally, calculate distances between specific landmarks, e.g., thumb to index tip
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    distance_thumb_index = np.sqrt((thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 + (thumb_tip.z - index_tip.z)**2)
    features = np.append(features, distance_thumb_index)

    return features

In [9]:
# mp_hands = mp.solutions.hands
# hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

# Dictionary to store features with indexes from `words_corpus_df`
features_dict = {}

for i in range(len(words_corpus_df)):
    full_path = words_corpus_df.iloc[i][1]
    # full_path = rf"{file_url}\{frame_path}"

    # Read the image with the correct full path
    image = cv2.imread(full_path)

    if image is not None:
        # Convert the image to RGB as MediaPipe works with RGB images
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        # Process the image with MediaPipe
        results = hands.process(image_rgb)
    
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extract features from each detected hand
                features = extract_features(hand_landmarks)
                # Store the features with the index
                features_dict[i] = features
    
        else:
            print(f"No hand landmarks detected in the image at index {i}")

# Convert the dictionary to a DataFrame to retain original indexes
features_df = pd.DataFrame.from_dict(features_dict, orient='index')

# Release MediaPipe resources
hands.close()


No hand landmarks detected in the image at index 2
No hand landmarks detected in the image at index 6
No hand landmarks detected in the image at index 7
No hand landmarks detected in the image at index 8
No hand landmarks detected in the image at index 15
No hand landmarks detected in the image at index 16
No hand landmarks detected in the image at index 17
No hand landmarks detected in the image at index 20
No hand landmarks detected in the image at index 25
No hand landmarks detected in the image at index 26
No hand landmarks detected in the image at index 28
No hand landmarks detected in the image at index 29
No hand landmarks detected in the image at index 30
No hand landmarks detected in the image at index 32
No hand landmarks detected in the image at index 33
No hand landmarks detected in the image at index 34
No hand landmarks detected in the image at index 35
No hand landmarks detected in the image at index 36
No hand landmarks detected in the image at index 37
No hand landmark

In [10]:
print(features_df.shape)

(1341, 64)


In [11]:
features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.310631,0.542717,5.756007e-08,0.309640,0.513490,-0.011111,0.302574,0.484154,-0.017704,0.299331,...,0.254004,0.554325,-0.022196,0.244248,0.556488,-0.025712,0.235559,0.557633,-0.028368,0.088687
1,0.349756,0.312070,1.188294e-07,0.361742,0.290425,-0.011149,0.369796,0.255245,-0.017230,0.376856,...,0.298417,0.236371,-0.018525,0.297799,0.226833,-0.019406,0.297672,0.220902,-0.019529,0.073274
3,0.377176,0.225823,9.052137e-08,0.389961,0.201429,-0.008178,0.398690,0.171779,-0.012374,0.406451,...,0.335104,0.133429,-0.014946,0.329040,0.116754,-0.017168,0.324146,0.101783,-0.018701,0.098201
4,0.628054,0.341598,1.404121e-07,0.614516,0.323601,-0.008881,0.604411,0.294863,-0.014143,0.594925,...,0.663216,0.258079,-0.021965,0.670425,0.244315,-0.024530,0.677098,0.231548,-0.026120,0.107741
5,0.624370,0.373200,1.460090e-07,0.611975,0.358099,-0.011499,0.603743,0.332259,-0.018976,0.595149,...,0.662842,0.312013,-0.025383,0.669632,0.301733,-0.028208,0.675252,0.292884,-0.029933,0.094570


In [12]:
features_df.tail()
# as last index is as same as as original so we can work with it

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
3187,0.516384,0.934833,1.200309e-07,0.502227,0.941857,-0.007903,0.490989,0.966455,-0.009668,0.483381,...,0.500649,1.025498,0.003856,0.493630,1.035574,0.003876,0.488021,1.041036,0.004341,0.040912
3188,0.466291,0.499750,4.077212e-08,0.490007,0.485130,-0.007605,0.509123,0.470275,-0.015650,0.514297,...,0.458378,0.503234,-0.036172,0.463337,0.510462,-0.033446,0.462120,0.504638,-0.031439,0.100023
3189,0.438552,0.430639,2.417242e-08,0.452326,0.422841,-0.009162,0.466148,0.410837,-0.017689,0.470169,...,0.414486,0.411160,-0.033348,0.425129,0.428160,-0.030595,0.431474,0.434662,-0.027218,0.106812
3190,0.188927,0.414196,3.298646e-07,0.217849,0.404225,-0.019165,0.263034,0.383194,-0.028543,0.296865,...,0.244951,0.375255,-0.057221,0.238747,0.387291,-0.054817,0.228009,0.397775,-0.048664,0.076164
3191,0.467205,0.484764,1.373348e-07,0.478713,0.443635,0.000470,0.498948,0.413178,-0.005146,0.518559,...,0.513509,0.461524,-0.031306,0.508589,0.467270,-0.025416,0.498717,0.466678,-0.020911,0.058348


In [13]:
word_column_df = words_corpus_df[['Word']]

combined_df = pd.concat([word_column_df, features_df], axis=1)

combined_df.head()

,Word,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,A LOT,0.310631,0.542717,5.756007e-08,0.309640,0.513490,-0.011111,0.302574,0.484154,-0.017704,...,0.254004,0.554325,-0.022196,0.244248,0.556488,-0.025712,0.235559,0.557633,-0.028368,0.088687
1,A LOT,0.349756,0.312070,1.188294e-07,0.361742,0.290425,-0.011149,0.369796,0.255245,-0.017230,...,0.298417,0.236371,-0.018525,0.297799,0.226833,-0.019406,0.297672,0.220902,-0.019529,0.073274
2,A LOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A LOT,0.377176,0.225823,9.052137e-08,0.389961,0.201429,-0.008178,0.398690,0.171779,-0.012374,...,0.335104,0.133429,-0.014946,0.329040,0.116754,-0.017168,0.324146,0.101783,-0.018701,0.098201
4,A LOT,0.628054,0.341598,1.404121e-07,0.614516,0.323601,-0.008881,0.604411,0.294863,-0.014143,...,0.663216,0.258079,-0.021965,0.670425,0.244315,-0.024530,0.677098,0.231548,-0.026120,0.107741


In [14]:
combined_df.shape

(3192, 65)

In [15]:
combined_df.isna().sum()

Word       0
0       1851
1       1851
2       1851
3       1851
        ... 
59      1851
60      1851
61      1851
62      1851
63      1851
Length: 65, dtype: int64

In [16]:
combined_df.dropna(axis=0, inplace=True)

In [17]:
combined_df.isna().sum()

Word    0
0       0
1       0
2       0
3       0
       ..
59      0
60      0
61      0
62      0
63      0
Length: 65, dtype: int64

In [18]:
combined_df.shape

(1341, 65)

In [19]:
cleaned_df = combined_df.copy()

In [20]:
# Saving cleaned_df to a CSV file
# preserving the indexes
cleaned_df.to_csv('words_data_new.csv', index=True)